# 분류예측의 불확실성 추정

어떤 테스트 포인트에 대ㅐ 분류기가 예측한 클래스가 무엇인지 뿐만 아니라 정확한 클래스임을 얼마나 확신하는지 중요할 때가 많다. 실제 어플리케이션에서는 오류의 종류에 따라 전혀 다른 결과를 만든다. 

scikit-learn 분류기에는 불확실성을 추정할 수 있는 함수가 두개가 있다. decision_function과 predict_proba이다. 

GradientBoostingClassifier 분류기의 decision_function과 predict_proba 메서드가 어떤 역할을 하는지 살펴보자.

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
import numpy as np
x,y = make_circles(noise = 0.25, factor =0.5, random_state=1)
y_named = np.array(['blue','red'])[y]

In [9]:
x_train,x_test,y_train_named,y_test_named,y_train,y_test = train_test_split(x,y_named,y,random_state=0)

In [10]:
gbrt =GradientBoostingClassifier(random_state=0)
gbrt.fit(x_train,y_train_named)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

## 결정함수

이진 분류에서 decision function 반환값의 크기는 n_sample이며 각 샘플이 하나의 실수 값을 반환한다.

In [15]:
print(x_test.shape)
print(gbrt.decision_function(x_test[:6]))

(25, 2)
[ 4.13592629 -1.7016989  -3.95106099 -3.62599351  4.28986668  3.66166106]


이 값은 모델이 데이터 포인트가 양성 클래스인 1에 속한다고 믿는 정도이다. 양수값은 양성 클래스를 의마하며 음수 값음 음수 클래스를 의미한다.

In [16]:
print("임계치와 결정 함수 결과 비교:\n{}".format(gbrt.decision_function(x_test) > 0))
print("예측:\n{}".format(gbrt.predict(x_test)))

임계치와 결정 함수 결과 비교:
[ True False False False  True  True False  True  True  True False  True
  True False  True False False False  True  True  True  True  True False
 False]
예측:
['red' 'blue' 'blue' 'blue' 'red' 'red' 'blue' 'red' 'red' 'red' 'blue'
 'red' 'red' 'blue' 'red' 'blue' 'blue' 'blue' 'red' 'red' 'red' 'red'
 'red' 'blue' 'blue']


이진분류에서 음성 클래스는 항상 classes_ 속성의 처 번째 원소이고 양성 클래스는 classes_의 두 번째 원소이다. 그래서 predict 함수의 결과를 완전히 재현하려면 classes_ 속성을 사용하면 된다.

In [21]:
#불리언 값을 0과1로 변환
greater_zero = (gbrt.decision_function(x_test) > 0).astype(int)

In [25]:
pred = gbrt.classes_[greater_zero]
pred

array(['red', 'blue', 'blue', 'blue', 'red', 'red', 'blue', 'red', 'red',
       'red', 'blue', 'red', 'red', 'blue', 'red', 'blue', 'blue', 'blue',
       'red', 'red', 'red', 'red', 'red', 'blue', 'blue'], 
      dtype='<U4')

In [26]:
print('pred는 예측 결과와 같다: {}'.format(np.all(pred == gbrt.predict(x_test))))

pred는 예측 결과와 같다: True


## 예측확률

predict_proba의 출력은 각 클래스에 대한 확률이다.

In [27]:
print("확률 값의 형테: {}".format(gbrt.predict_proba(x_test).shape))

확률 값의 형테: (25, 2)


각 행의 첫 번째 원소는 첫 번째 클래스의 예측 확률이고 두 번째 원소는 두 번쨰 클래스의 예측 확률이다.Predict_proba의 출력은 항상 0과 1사이의 값이며 두 클래스에 대한 확률의 합은 항상1이다.

In [30]:
print("예측확률:\n{}".format(gbrt.predict_proba(x_test[:6])))

예측확률:
[[ 0.01573626  0.98426374]
 [ 0.84575649  0.15424351]
 [ 0.98112869  0.01887131]
 [ 0.97406775  0.02593225]
 [ 0.01352142  0.98647858]
 [ 0.02504637  0.97495363]]


두 클래스의 확률의 합은 1이므로 두 클래스 중 하나는 50# 이상의 확신을 가질 것이 틀림없다. 그리고 바로 그 클래스가 예측값이 된다.

## 다중 분류에서의 불확실성

decision_function과 predict_proba 메서드는 다중 분류에서도 사용할 수 있다. 클래스가 세 개인 iris 데이터셋에 적용해보자

In [33]:
from sklearn.datasets import load_iris
iris = load_iris()

x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state = 42)

gbrt = GradientBoostingClassifier(learning_rate=0.01,random_state=0)
gbrt.fit(x_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

### decsion_function

In [38]:
print("결정 함수의  결과 형태:{}".format(gbrt.decision_function(x_test).shape))

결정 함수의  결과 형태:(38, 3)


In [40]:
print("결정 함수 결과:\n{}".format(gbrt.decision_function(x_test)[:6,:]))

결정 함수 결과:
[[-0.52931069  1.46560359 -0.50448467]
 [ 1.51154215 -0.49561142 -0.50310736]
 [-0.52379401 -0.4676268   1.51953786]
 [-0.52931069  1.46560359 -0.50448467]
 [-0.53107259  1.28190451  0.21510024]
 [ 1.51154215 -0.49561142 -0.50310736]]


각 열은 각 클래스에 대한 확신 점수를 담고 있다. 수치가 크면 그 클래스일 가능성이 크고 수치가 작으면 그 클래스일 가능성이 낮다는 것이다. 데이터 포인트마다 점수들에서 가장 큰 값을 찾아 예측 결과를 재현할 수 있다.

In [45]:
print("가장 큰 결정 함수의 인덱스:\n{}".format(np.argmax(gbrt.decision_function(x_test),axis=1)))
print("예측:\n{}".format(gbrt.predict(x_test)))

가장 큰 결정 함수의 인덱스:
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]
예측:
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


### predict_proba

In [48]:
print("예측확률:\n{}".format(gbrt.predict_proba(x_test)[:6]))
print("합:{}".format(gbrt.predict_proba(x_test)[:6].sum(axis=1)))

예측확률:
[[ 0.10664722  0.7840248   0.10932798]
 [ 0.78880668  0.10599243  0.10520089]
 [ 0.10231173  0.10822274  0.78946553]
 [ 0.10664722  0.7840248   0.10932798]
 [ 0.10825347  0.66344934  0.22829719]
 [ 0.78880668  0.10599243  0.10520089]]
합:[ 1.  1.  1.  1.  1.  1.]


In [50]:
print("가장 큰 예측확률의 인덱스:\n{}".format(np.argmax(gbrt.predict_proba(x_test),axis=1)))
print("에측:\n{}".format(gbrt.predict(x_test)))

가장 큰 예측확률의 인덱스:
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]
에측:
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]
